# 🔌 Connecting API

First, we set up the connection to the language model using **litellm**. This step imports the required libraries, configures the API key, and initializes the unified interface that allows communication with different LLM providers through a single API. The code also includes a validation step that automatically detects whether a Vocareum key is present and, if so, configures the appropriate API endpoint.

In [ ]:
import litellm as lt
import os
from dotenv import load_dotenv
load_dotenv()


if os.getenv("OPENAI_API_KEY"):
    lt.openai_key = os.getenv("OPENAI_API_KEY")

if (lt.openai_key or "").startswith("voc-"):
    lt.api_base = "https://openai.vocareum.com/v1"
    print("Detected vocareum API key. Using Vocareum OpenAI API base URL.")

Our first major step is to delegate the implementation of the factorial function to the language model. To achieve this, we define two complementary prompts. The `SYSTEM_PROMPT` establishes the role and behavioral constraints of the LLM (e.g., acting as a precise and reliable code generator), while the `USER_PROMPT` specifies the concrete functional requirements, such as input constraints, expected behavior, and output format. Together, these prompts guide the model to generate a correct and well-structured implementation of the factorial function aligned with our specifications.

In [ ]:
SYSTEM_PROMPT = """You are a helpful coding assistant. Return only the Python code, with no explanation or preamble."""
USER_PROMPT = """Write a Python function that implements the factorial algorithm using recursion.
The name of the function should be `factorial`.
The function should take a single non-negative integer as input.
Include a docstring explaining what the function does."""

Now, we'll send these prompts to the model. We use the completion function from the **litellm** library, specifying the model **gpt-5-mini** and passing our prompts. Then we print the generated code from the response.

In [ ]:
from litellm import completion

response = completion(
    model="gpt-5-nano",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT},
    ],
)
print("Generated Code:\n")
print(response.choices[0].message.content)

The LLM has generated a Python function that implements the recursive logic and also includes type and value checking for the input, along with a detailed docstring, just as we asked.

In [ ]:
def factorial(n):
    """Return the factorial of a non-negative integer n computed recursively.

    The factorial of n (written n!) is the product of all positive integers
    up to n. By definition, 0! == 1.

    Args:
        n (int): A non-negative integer.

    Returns:
        int: The factorial of n.

    Raises:
        TypeError: If n is not an integer.
        ValueError: If n is negative.
    """
    if not isinstance(n, int):
        raise TypeError("n must be an integer")
    if n < 0:
        raise ValueError("n must be non-negative")
    if n == 0 or n == 1:
        return 1
    return n * factorial(n - 1)

With the function defined, we can perform a quick check to see if it works correctly. We'll test it with a simple case: the factorial of 5, which should be 120.

In [ ]:
input_val = 5
expected_val = 120

output_val = factorial(input_val)

print(f"Input: {input_val}")
print(f"Output: {output_val}")
print(f"Expected: {expected_val}")
print(f"Match: {output_val == expected_val}")
print()

if output_val == expected_val:
    print("Test case passed! ✅")
else:
    print("Test case failed. ❌")

A single manual check is good, but a full test suite is better. Now, we'll ask the LLM to generate a comprehensive set of tests using Python's built-in unittest framework. We'll specifically ask it to cover a basic positive integer, the edge case of 0, and an invalid negative number.

In [ ]:
SYSTEM_PROMPT = """You are a helpful coding assistant that writes unit tests using the unittest framework. Return only the code, no explanations."""
USER_PROMPT = """Please generate test cases for the `factorial` function.
The test cases should be in a class called `TestFactorial` that inherits from `unittest.TestCase`.
Include tests for a positive integer, the number zero, and a negative number.
The test for the negative number should check that a `ValueError` is raised.
Do not include any imports or a main block.
"""

We send this new prompt to the model.

In [ ]:
response = completion(
    model="gpt-5-mini",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT},
    ],
)
print("Generated Test Cases:\n")
print(response.choices[0].message.content)

The LLM generates the Python code for a test class complete with three distinct test methods that cover our requested scenarios.

For the final step, we'll take the LLM-generated test code, place it in our notebook, and run it to formally validate our factorial function.

In [ ]:
import unittest

class TestFactorial(unittest.TestCase):
    def test_positive_integer(self):
        self.assertEqual(factorial(5), 120)

    def test_zero(self):
        self.assertEqual(factorial(0), 1)

    def test_negative_raises(self):
        with self.assertRaises(ValueError):
            factorial(-3)

# Run the tests
unittest.main(argv=["notebook-tests"], exit=False)

The output confirms that all three tests ran and passed, giving us confidence that our LLM-generated function is behaving as expected.

By crafting clear and specific prompts, you can leverage an LLM to automate the generation of both functional code and its corresponding validation tests.

